In [22]:
import pandas as pd
import os
import numpy as np
from dateutil import parser
import logging
from datetime import datetime
import re
import time

In [23]:
def get_xlsx_data(directory):
    # 检查目录是否存在
    if not os.path.isdir(directory):
        print("提供的路径不是一个有效的目录")
        return

    # 初始化一个空的 DataFrame 来存储所有文件的数据
    all_data = pd.DataFrame()
    num_of_all_files = 0
    num_of_targeted_files = 0
    skip_files_list = []

    # 遍历目录及其子目录中的所有 Excel 文件
    for root, dirs, files in os.walk(directory):
        
        last_folder_name = os.path.basename(root)
        
        for filename in files:
            if filename.endswith(".xlsx"):
                num_of_all_files += 1
            
            if filename.endswith('数据开放.xlsx'):
                file_path = os.path.join(root, filename)
                print(f"获取 {filename} 成功")
                num_of_targeted_files += 1
                # 读取 Excel 文件
                df = pd.read_excel(file_path)

                # 增加省份名称
                parts = filename.split('_')
                df['location'] = parts[0]
                
                df['province'] = last_folder_name
                
                # 添加数据到总 DataFrame
                all_data = pd.concat([all_data, df], ignore_index=True)
            elif filename.endswith('.xlsx'): 
                skip_files_list.append(filename)
                
    # 检查必要的列是否存在
    required_columns = ['location', 'title', 'subject', 'description', 'source_department',
                        'release_time', 'update_time', 'open_conditions', 'data_volume',
                        'is_api', 'file_type', 'access_count', 'download_count',
                        'api_call_count', 'link', 'update_cycle']
    if not all(col in all_data.columns for col in required_columns):
        print("某些必要列缺失")
        return
    
    print(f"共获取 {num_of_targeted_files} 个有效文件，共 {num_of_all_files} 个文件")
    print(f"跳过 {len(skip_files_list)} 个文件：{skip_files_list}")
    return all_data

In [24]:
# 建立更新周期与每年更新次数的映射关系
frequency_map = {
        '每年': 1, '按年': 1, '按年更新': 1, '1年': 1, '年': 1, '一年': 1, '年年': 1, '12个月': 1, '每年进行一次评选': 1,
        '年度': 1, '一年更新一次': 1, '每年统计一次': 1, '每年一次': 1, '每年进行一次': 1, '每年一次更新': 1, '每年更新一次': 1, '每年认定一次': 1, 
        '每年更新': 1,
        '每半年': 2, '半年更新': 2, '6个月': 2, '六个月': 2, '半年': 2, 
        '2年': 1/2, '二年': 1/2, '每两年': 1/2, '每两年评选一次': 1/2,
        '每三年': 1/3, '三年': 1/3, '3年': 1/3, '每三年评选一次': 1/3,
        '每五年': 1/5, '5年': 1/5,
        '每十年': 1/10, '10年': 1/10,
        '每季度': 4, '按季度': 4, '每季': 4, '按季': 4, '按季度更新': 4, '按季更新': 4, '季度': 4, '3个月': 4, '每季度更新': 4,
        '四个月': 12/4,
        '两个月': 12 / 2,
        '每月': 12, '按月': 12, '按月更新': 12, '月': 12,
        '每周': 52, '按周': 52, '按周更新': 52,
        '实时': 365, '即时': 365, '每日': 365, '按日更新': 365, '每天': 365, '按天': 365, '实时更新': 365, '按天更新': 365, '按日': 365, '天': 365,
        '按小时': 24 * 30, '小时级': 24 * 30,
        '按分钟': 60 * 24 * 30, '分钟级': 60 * 24 * 30,
        '其他': 0, '未定义': 0, '自定义': 0, '静态数据': 0, '静态': 0, '静态文件不更新': 0, '数据截至2020年': 0, '不在此平台更新': 0,
        '不在此平台发布更新': 0, '历史数据不更新': 0, '无需更新': 0, '无变化不更新': 0,
        '用户自定义': 0, '有变动时更新': 0, '不定期，有变动时更新': 0, '有变化时更新': 0, '不定期更新': 0, '按需更新': 0, '工业总量不对外': 0, '投资总量数据不对外': 0, '有更新的信息就更新': 0, '有新的类型再进行更新': 0, '人口普查工作开展时更': 0,
        '每次普查结果实时更新': 0, '申请维修支出时查询': 0, '有案件更新时更新数据': 0, 
        '高院发布数据时更新': 0, '有普查更新时更新': 0, '普查时更新': 0, '有更新时更新': 0, '发生实际变更时': 0, '按批复文进行更新': 0, '根据实际情况': 0, '以后由应急管理局发': 0,'未知': 0, '适时': 0, '信用承诺': 0, '不定期': 0, '不更新': 0, '其它': 0, '不定时': 0,
        '无':0, '不可交换': 0, 
    }

city_dict = {
    "北京市": 1,
    "天津市": 2,
    "石家庄市": 3,
    "唐山市": 4,
    "秦皇岛市": 5,
    "邯郸市": 6,
    "邢台市": 7,
    "保定市": 8,
    "张家口市": 9,
    "承德市": 10,
    "沧州市": 11,
    "廊坊市": 12,
    "衡水市": 13,
    "太原市": 14,
    "大同市": 15,
    "阳泉市": 16,
    "长治市": 17,
    "晋城市": 18,
    "朔州市": 19,
    "晋中市": 20,
    "运城市": 21,
    "忻州市": 22,
    "临汾市": 23,
    "吕梁市": 24,
    "呼和浩特市": 25,
    "包头市": 26,
    "乌海市": 27,
    "赤峰市": 28,
    "通辽市": 29,
    "鄂尔多斯市": 30,
    "呼伦贝尔市": 31,
    "巴彦淖尔市": 32,
    "乌兰察布市": 33,
    "沈阳市": 34,
    "大连市": 35,
    "鞍山市": 36,
    "抚顺市": 37,
    "本溪市": 38,
    "丹东市": 39,
    "锦州市": 40,
    "营口市": 41,
    "阜新市": 42,
    "辽阳市": 43,
    "盘锦市": 44,
    "铁岭市": 45,
    "朝阳市": 46,
    "葫芦岛市": 47,
    "长春市": 48,
    "吉林市": 49,
    "四平市": 50,
    "辽源市": 51,
    "通化市": 52,
    "白山市": 53,
    "松原市": 54,
    "白城市": 55,
    "哈尔滨市": 56,
    "齐齐哈尔市": 57,
    "鸡西市": 58,
    "鹤岗市": 59,
    "双鸭山市": 60,
    "大庆市": 61,
    "伊春市": 62,
    "佳木斯市": 63,
    "七台河市": 64,
    "牡丹江市": 65,
    "黑河市": 66,
    "绥化市": 67,
    "上海市": 68,
    "南京市": 69,
    "无锡市": 70,
    "徐州市": 71,
    "常州市": 72,
    "苏州市": 73,
    "南通市": 74,
    "连云港市": 75,
    "淮安市": 76,
    "盐城市": 77,
    "扬州市": 78,
    "镇江市": 79,
    "泰州市": 80,
    "宿迁市": 81,
    "杭州市": 82,
    "宁波市": 83,
    "温州市": 84,
    "嘉兴市": 85,
    "湖州市": 86,
    "绍兴市": 87,
    "金华市": 88,
    "衢州市": 89,
    "舟山市": 90,
    "台州市": 91,
    "丽水市": 92,
    "合肥市": 93,
    "芜湖市": 94,
    "蚌埠市": 95,
    "淮南市": 96,
    "马鞍山市": 97,
    "淮北市": 98,
    "铜陵市": 99,
    "安庆市": 100,
    "黄山市": 101,
    "滁州市": 102,
    "阜阳市": 103,
    "宿州市": 104,
    "六安市": 105,
    "亳州市": 106,
    "池州市": 107,
    "宣城市": 108,
    "福州市": 109,
    "厦门市": 110,
    "莆田市": 111,
    "三明市": 112,
    "泉州市": 113,
    "漳州市": 114,
    "南平市": 115,
    "龙岩市": 116,
    "宁德市": 117,
    "南昌市": 118,
    "景德镇市": 119,
    "萍乡市": 120,
    "九江市": 121,
    "新余市": 122,
    "鹰潭市": 123,
    "赣州市": 124,
    "吉安市": 125,
    "宜春市": 126,
    "抚州市": 127,
    "上饶市": 128,
    "济南市": 129,
    "青岛市": 130,
    "淄博市": 131,
    "枣庄市": 132,
    "东营市": 133,
    "烟台市": 134,
    "潍坊市": 135,
    "济宁市": 136,
    "泰安市": 137,
    "威海市": 138,
    "日照市": 139,
    "临沂市": 140,
    "德州市": 141,
    "聊城市": 142,
    "滨州市": 143,
    "菏泽市": 144,
    "郑州市": 145,
    "开封市": 146,
    "洛阳市": 147,
    "平顶山市": 148,
    "安阳市": 149,
    "鹤壁市": 150,
    "新乡市": 151,
    "焦作市": 152,
    "濮阳市": 153,
    "许昌市": 154,
    "漯河市": 155,
    "三门峡市": 156,
    "南阳市": 157,
    "商丘市": 158,
    "信阳市": 159,
    "周口市": 160,
    "驻马店市": 161,
    "武汉市": 162,
    "黄石市": 163,
    "十堰市": 164,
    "宜昌市": 165,
    "襄阳市": 166,
    "鄂州市": 167,
    "荆门市": 168,
    "孝感市": 169,
    "荆州市": 170,
    "黄冈市": 171,
    "咸宁市": 172,
    "随州市": 173,
    "长沙市": 174,
    "株洲市": 175,
    "湘潭市": 176,
    "衡阳市": 177,
    "邵阳市": 178,
    "岳阳市": 179,
    "常德市": 180,
    "张家界市": 181,
    "益阳市": 182,
    "郴州市": 183,
    "永州市": 184,
    "怀化市": 185,
    "娄底市": 186,
    "广州市": 187,
    "韶关市": 188,
    "深圳市": 189,
    "珠海市": 190,
    "汕头市": 191,
    "佛山市": 192,
    "江门市": 193,
    "湛江市": 194,
    "茂名市": 195,
    "肇庆市": 196,
    "惠州市": 197,
    "梅州市": 198,
    "汕尾市": 199,
    "河源市": 200,
    "阳江市": 201,
    "清远市": 202,
    "东莞市": 203,
    "中山市": 204,
    "潮州市": 205,
    "揭阳市": 206,
    "云浮市": 207,
    "南宁市": 208,
    "柳州市": 209,
    "桂林市": 210,
    "梧州市": 211,
    "北海市": 212,
    "防城港市": 213,
    "钦州市": 214,
    "贵港市": 215,
    "玉林市": 216,
    "百色市": 217,
    "贺州市": 218,
    "河池市": 219,
    "来宾市": 220,
    "崇左市": 221,
    "海口市": 222,
    "三亚市": 223,
    "三沙市": 224,
    "儋州市": 225,
    "重庆市": 226,
    "成都市": 227,
    "自贡市": 228,
    "攀枝花市": 229,
    "泸州市": 230,
    "德阳市": 231,
    "绵阳市": 232,
    "广元市": 233,
    "遂宁市": 234,
    "内江市": 235,
    "乐山市": 236,
    "南充市": 237,
    "眉山市": 238,
    "宜宾市": 239,
    "广安市": 240,
    "达州市": 241,
    "雅安市": 242,
    "巴中市": 243,
    "资阳市": 244,
    "贵阳市": 245,
    "六盘水市": 246,
    "遵义市": 247,
    "安顺市": 248,
    "毕节市": 249,
    "铜仁市": 250,
    "昆明市": 251,
    "曲靖市": 252,
    "玉溪市": 253,
    "保山市": 254,
    "昭通市": 255,
    "丽江市": 256,
    "普洱市": 257,
    "临沧市": 258,
    "拉萨市": 259,
    "日喀则市": 260,
    "昌都市": 261,
    "林芝市": 262,
    "山南市": 263,
    "那曲市": 264,
    "西安市": 265,
    "铜川市": 266,
    "宝鸡市": 267,
    "咸阳市": 268,
    "渭南市": 269,
    "延安市": 270,
    "汉中市": 271,
    "榆林市": 272,
    "安康市": 273,
    "商洛市": 274,
    "兰州市": 275,
    "嘉峪关市": 276,
    "金昌市": 277,
    "白银市": 278,
    "天水市": 279,
    "武威市": 280,
    "张掖市": 281,
    "平凉市": 282,
    "酒泉市": 283,
    "庆阳市": 284,
    "定西市": 285,
    "陇南市": 286,
    "西宁市": 287,
    "海东市": 288,
    "银川市": 289,
    "石嘴山市": 290,
    "吴忠市": 291,
    "固原市": 292,
    "中卫市": 293,
    "乌鲁木齐市": 294,
    "克拉玛依市": 295,
    "吐鲁番市": 296,
    "哈密市": 297
}

set_undefined = {}

open_condition_undefined = {}


def process_location(location):
     # 确保 location 是一个字符串
    if not isinstance(location, str):
        print(location, type(location))
        return location
    
    # 如果存在 '_'，只取 '_' 前面的部分
    if '_' in location:
        location = location.split('_')[0]
    # 如果最后一个字不是 '市'，则添加 '市'
    if not location.endswith('市') and not location.endswith('区') and not location.endswith('省'):
        location += '市'
    return location

def convert_data_volume(volume):
    """
    将不同格式的数据容量转换为统一的整数形式，单位为MB。

    参数:
    volume (str, float, or int): 原始数据容量字符串或数值，如 "1G", "0.5", "30M" 或数字。

    返回:
    int: 转换后的数据容量整数，若为 "若干" 则返回 1，无法转化为整数则返回 0。
    """
    if volume is None or volume == '':
        return None  # 保留为空
    try:
        if isinstance(volume, str):  # 如果volume是字符串，执行单位转换
            volume = volume.strip()  # 去除字符串两端的空格
            if volume == '若干':
                return 1  # 转换为1
            if 'G' in volume:
                return int(float(volume.upper().replace('G', '')) * 1024)  # 转换为MB，转为整数
            elif 'M' in volume:
                return int(float(volume.upper().replace('M', '')))  # 去除M，转换为整数
            else:
                return int(float(volume))  # 没有单位，转换为浮点数后转为整数
        elif isinstance(volume, (float, int)):  # 检查volume是否为数值（包括float和int）
            return int(volume)  # 转换为整数
    except ValueError as e:
        return None  # 返回0，避免在转换为整数时出现错误

    return None  # 返回0，避免没有匹配到任何情况时的错误

# 将 'is_api' 字段转换为布尔值
def convert_to_bool(value):
    if value is None or value == '':
        return None
    
    if isinstance(value, str):
        return value.lower() == 'true'
    return bool(value)

def calculate_update_frequency(row):
    """
    根据更新周期计算每月更新频率。

    参数:
    row (Series): 包含了需要的更新信息的行数据。

    返回:
    float: 每月更新次数。
    """
    update_cycle = row['update_cycle']
    # 若更新周期为空，则返回None
    if update_cycle is None or update_cycle == '' :
        return None
    
    release_time = pd.to_datetime(row['release_time'], errors='coerce')
    update_time = pd.to_datetime(row['update_time'], errors='coerce')

    # 如果发布时间晚于更新时间，则不更新
    if release_time >= update_time:
        return None
    
    # 如果更新周期不是字符串，直接返回0
    if not isinstance(update_cycle, str):
        return None
    
    update_cycle = update_cycle.replace("‘", '').replace("’",'').replace('自定义：', '').strip()

    
    if update_cycle not in frequency_map:
        update_cycle = str(update_cycle)
        if update_cycle not in set_undefined:
            set_undefined[update_cycle] = 1
        else:
            set_undefined[update_cycle] += 1
        return 0

    # 若更新周期在映射字典中，则返回每月更新次数，否则返回0
    return frequency_map.get(update_cycle, None) / 12

def calculate_timed_update_rate(row):
    """
    根据更新周期计算每月更新频率。

    参数:
    row (Series): 包含了需要的更新信息的行数据。

    返回:
    float: 每月更新次数。
    """
    update_cycle = row['update_cycle']
    # 若更新周期为空，则返回None
    if update_cycle is None or update_cycle == '' or not isinstance(update_cycle, str):
        return None
    
    update_cycle = update_cycle.replace("‘", '').replace("’",'').replace('自定义：', '').strip()

    
    if update_cycle not in frequency_map:
        update_cycle = str(update_cycle)
        if update_cycle not in set_undefined:
            set_undefined[update_cycle] = 1
        else:
            set_undefined[update_cycle] += 1
        return 0

    # 若更新周期在映射字典中，则返回每月更新次数，否则返回0
    return frequency_map.get(update_cycle, 0)
    

def calculate_is_update(row):
    """
    计算更新频率。

    参数:
    row (Series): DataFrame中的一行。

    返回:
    bool: 指示文本是否有更新。
    """
    release_time = pd.to_datetime(row['release_time'], errors='coerce')
    update_time = pd.to_datetime(row['update_time'], errors='coerce')

    # 如果发布时间晚于更新时间，则不更新
    if pd.notnull(release_time) and pd.notnull(update_time) and release_time < update_time:
        return True
    elif pd.isnull(release_time) or pd.isnull(update_time):
        return None
    else:
        return False

def safe_parse_datetime(date_str):
    """尝试解析日期字符串，兼容多种格式，如果失败则返回 pd.NaT."""
    try:
        # dateutil.parser.parse 更加灵活，可以解析不完全一致的日期格式
        return parser.parse(date_str)
    except ValueError:
        return pd.NaT

def remove_chinese_punctuation(text):
    """去除字符串中的中文标点符号"""
    chinese_punctuation = r'[\u3000-\u303F\uFF00-\uFFEF]'
    cleaned_text = re.sub(chinese_punctuation, '', text)
    return cleaned_text

def map_open_conditions(value):
    if pd.isnull(value) or value == '' or value == 'nan':
        return None
    
    if isinstance(value, str):
        value = remove_chinese_punctuation(value)
    
    if value not in open_conditions_mapping:
        
        if value not in open_condition_undefined:
            open_condition_undefined[value] = 1
        else:
            open_condition_undefined[value] += 1
            
        return value
    return open_conditions_mapping[value]


In [25]:
# 使用示例
folder_path = r'../../docs/after_cities_output_files'
output_path = r'../../docs'
df = get_xlsx_data(folder_path)

获取 上海市_数据开放.xlsx 成功
获取 乌兰察布市_数据开放.xlsx 成功
获取 呼和浩特市_数据开放.xlsx 成功
获取 北京市_数据开放.xlsx 成功
获取 乐山市_数据开放.xlsx 成功
获取 南充市_数据开放.xlsx 成功
获取 巴中市_数据开放.xlsx 成功
获取 广元市_数据开放.xlsx 成功
获取 广安市_数据开放.xlsx 成功
获取 德阳市_数据开放.xlsx 成功
获取 成都市_数据开放.xlsx 成功
获取 泸州市_数据开放.xlsx 成功
获取 眉山市_数据开放.xlsx 成功
获取 绵阳市_数据开放.xlsx 成功
获取 自贡市_数据开放.xlsx 成功
获取 资阳市_数据开放.xlsx 成功
获取 达州市_数据开放.xlsx 成功
获取 遂宁市_数据开放.xlsx 成功
获取 雅安市_数据开放.xlsx 成功
获取 天津市_数据开放.xlsx 成功
获取 吴忠市_数据开放.xlsx 成功
获取 宁夏省_数据开放.xlsx 成功
获取 石嘴山市_数据开放.xlsx 成功
获取 银川市_数据开放.xlsx 成功
获取 亳州市_数据开放.xlsx 成功
获取 六安市_数据开放.xlsx 成功
获取 合肥市_数据开放.xlsx 成功
获取 安徽省_数据开放.xlsx 成功
获取 宣城市_数据开放.xlsx 成功
获取 宿州市_数据开放.xlsx 成功
获取 池州市_数据开放.xlsx 成功
获取 淮北市_数据开放.xlsx 成功
获取 淮南市_数据开放.xlsx 成功
获取 滁州市_数据开放.xlsx 成功
获取 芜湖市_数据开放.xlsx 成功
获取 蚌埠市_数据开放.xlsx 成功
获取 铜陵市_数据开放.xlsx 成功
获取 马鞍山市_数据开放.xlsx 成功
获取 黄山市_数据开放.xlsx 成功
获取 东营市_数据开放.xlsx 成功
获取 临沂市_数据开放.xlsx 成功
获取 威海市_数据开放.xlsx 成功
获取 德州市_数据开放.xlsx 成功
获取 日照市_数据开放.xlsx 成功
获取 枣庄市_数据开放.xlsx 成功
获取 泰安市_数据开放.xlsx 成功
获取 济南市_数据开放.xlsx 成功
获取 济宁市_数据开放.xlsx 成功
获取 淄博市_数据开放.xlsx 成功
获取 滨州市_数据开放.xl

In [26]:
# df_test = df.copy()
# # 计算平均每月更新频率
# df_test['平均更新/月'] = df_test.apply(calculate_update_frequency, axis=1).round(2)

In [27]:
df_copy = df.copy()

df_copy['location'] = df_copy['location'].apply(process_location)

df_copy['data_volume'] = df_copy['data_volume'].apply(convert_data_volume)
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 数据容量转换完成")

# 计算平均每月更新频率
df_copy['定时更新率'] = df_copy.apply(calculate_timed_update_rate, axis=1).round(2)
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 定时更新率计算完成")
print("以下是未知的更新周期：", set_undefined)

# 计算是否更新列
df_copy['is_update'] = df_copy.apply(calculate_is_update, axis=1)
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 是否更新列计算完成")

# 确保release_time和update_time列是字符串类型
df_copy['release_time'] = df_copy['release_time'].astype(str)
df_copy['update_time'] = df_copy['update_time'].astype(str)

# 使用numpy.where选择release_time或update_time
# 将 release_time 为空字符串或者为字符串 "nan" 的情况下，使用 update_time 来填充 chosen_time
# 将 "nan" 替换为空字符串
df_copy['chosen_time'] = df_copy['release_time'].fillna('').replace('nan', '')
df_copy['chosen_time'] = np.where(df_copy['chosen_time'] == '', df_copy['update_time'], df_copy['chosen_time'])

# 应用自定义的安全日期解析函数，并提取年份
df_copy['year'] = df_copy['chosen_time'].apply(safe_parse_datetime).dt.year
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 年份列计算完成")

 # 将x映射为相应的开放条件
open_conditions_mapping = {
'无条件共享': '无条件开放', '一般公开': '无条件开放',
'无条件更新': '无条件开放', '普通公开简单注册用户可下载': '无条件开放', '普通公开,简单注册用户可下载': '无条件开放', 
'有条件公开': '有条件开放', '有开放条件': '有条件开放',
'无条件开放': '无条件开放',
'无条件': '无条件开放',
'登录开放': '无条件开放',
'普遍开放': '无条件开放',
'实名认证开放': '无条件开放',
'普通开放': '无条件开放',
'有条件': '有条件开放',
'有条件开放': '有条件开放',
'依申请开放': '有条件开放',
'审核开放': '有条件开放',
'有条件共享': '有条件开放',
'实名注册用户可下载': '有条件开放',
'申请开放': '有条件开放',
'完全公开': '无条件开放',
'部门审批': '有条件开放',
'审批开放': '有条件开放',
'依申请': '有条件开放',
'依条件开放': '有条件开放',
'根据业务需求提供': '有条件开放',
'依申请公开': '有条件开放',
'已申请开放': '有条件开放',
'以申请开放': '有条件开放',
'申请': '有条件开放',
'需要用户授权': '有条件开放',
'完全开放': '无条件开放',
'半开放': '有条件开放',
'不开放': '无条件开放',
'开放': '无条件开放',
'参考': '无条件开放',
'需申请使用': '有条件开放',
'依具体申请原因审核是否予以开放': '有条件开放',
'脱敏开放': '有条件开放',
'平台审核': '有条件开放',
'申请审批': '有条件开放',
'平台审批': '有条件开放',
'需申请': '有条件开放',
'工作参考': '无条件开放',
'无条件开发': '无条件开放',
'工作所需': '无条件开放',
'申请后公开': '有条件开放',
'申请后开放': '有条件开放',
'无条件公开': '无条件开放',
'用于数据核验': '无条件开放',
'五条件共享': '无条件开放',  # 有疑问，请确认
'无条件开房': '无条件开放',  # 有疑问，请确认
'申请公开': '有条件开放',
'需要提供查询函': '有条件开放',
'行政依据': '无条件开放',
'无条件共享，工作参考': '无条件开放',
'须提供具体申请依据、实际用途和申请主体的相关信息。': '有条件开放',
'眉山市有效外观明细表': '无条件开放',  # 有疑问，请确认
'开放数据': '无条件开放',
'社会保障卡启用': '无条件开放',  # 有疑问，请确认
'办理施工许可证证可以申请查看': '有条件开放',
'安全监督备案表': '无条件开放',
'无条件开共享': '无条件开放',
'此项政务信息资源仅用于数据校核、业务协同。': '无条件开放',
'此数据仅限于工作参考和数据校核': '无条件开放',
'本数据仅限用于工作参考': '无条件开放',
'办公人员联络方式用于业务协调': '无条件开放',
'可共享': '无条件开放',
'无条件开放。': '无条件开放',
'市级部门无条件共享': '无条件开放',
'无条件共共享': '无条件开放',  # 有疑问，请确认
'眉山市路灯照明工程情况': '无条件开放',  # 有疑问，请确认
'有条件工程': '有条件开放',  # 有疑问，请确认
'有': '有条件开放',
'仅作工作参考': '无条件开放',
'数据仅用于业务办理查询': '有条件开放',
'数据仅用于业务办理': '有条件开放',
'数据仅用于业务办理和查询': '有条件开放',
'数据用于业务办理': '有条件开放',
'不予共享': '无条件开放',
'业务协同': '无条件开放',
'WU': '无条件开放',  # 有疑问，请确认
'仅用于工作参考': '无条件开放',
'数据仅用于共享业务查询': '有条件开放',
'农村计划生育家庭奖励扶助情况': '无条件开放',  # 有疑问，请确认
'独生子女父母专项奖励情况': '无条件开放',  # 有疑问，请确认
'共享': '无条件开放',
'用作工作参考': '无条件开放',
'依据：《中华人民共和国森林法》和《森林法实施条例》。': '无条件开放',  # 有疑问，请确认
'提供查询的函。': '有条件开放',
'用作业务协同': '无条件开放',
'仅对上级法院系统开放': '有条件开放',
'无条件g共享': '无条件开放',  # 有疑问，请确认
'依申请共享': '有条件开放',
'经申请': '有条件开放',
'该数据仅供参考，如有疑问请联系眉山市公安局天府新区分局。': '有条件开放',  # 有疑问，请确认
'公开数据': '无条件开放',
'有条件开发': '有条件开放',
'对所有人员公开': '无条件开放',
'已申请': '有条件开放',
'向社会开放': '无条件开放',
'无条件开放共享': '无条件开放',
'身份证申请或申请书': '有条件开放',
'眉山天府新区林区分布一览表': '无条件开放',  # 有疑问，请确认
'彭山区科普惠民共享基地名单': '无条件开放',  # 有疑问，请确认
'区科协开展科技下乡活动统计': '无条件开放',  # 有疑问，请确认
'彭山区农技协名单': '无条件开放',  # 有疑问，请确认
'先申请': '有条件开放',
'无条件共享类': '无条件开放',  # 有疑问，请确认
'与残联相关业务': '无条件开放',  # 有疑问，请确认
'无。': '无条件开放',  # 有疑问，请确认
'青神县融媒体中心广播电视编辑记者采编考试合格证持证情况': '无条件开放',  # 有疑问，请确认
'单位来函': '有条件开放',
'单位发函': '有条件开放',
'仅用于数据参考。': '无条件开放',
'仅用于数据参考': '无条件开放',
'仅用于工作参考。': '无条件开放',
'仅作为工作参考': '无条件开放',
'仅供工作参考': '无条件开放',
'仅用于工作参考、数据校核。': '无条件开放',
'仅用于工作参考，数据校核。': '无条件开放',
'用于数据校核、工作参考、业务协同。': '无条件开放',
'用于工作参考可申请共享': '有条件开放',
'用于工作需要可申请共享': '有条件开放',
'洪雅县外贸企业名单': '无条件开放',  # 有疑问，请确认
'作为工作参考。': '无条件开放',
'作为行政执法依据进行公开': '无条件开放',
'行政公开': '无条件开放',
'申请同意后开放': '有条件开放',
'无条件分享': '无条件开放',
'有条件分享': '有条件开放',
'文件': '无条件开放',
'无条件开放数据': '无条件开放',
'数据仅用于业务查询办理': '有条件开放',
'向社会公开': '无条件开放',
'对所有人公开': '无条件开放',
'审计法治工作报告': '无条件开开放',
'无条件是使用，用于开展正常工作': '无条件开放',
'仅用于工作参考，业务协同。': '无条件开放',
'洪雅县地方政府债务限额汇总表': '无条件开放',  # 有疑问，请确认
'公务用车购置及运行费统计表': '无条件开放',  # 有疑问，请确认
'有条件公共享': '有条件开放',  # 有疑问，请确认
'彭山称号企业汇总': '无条件开放',  # 有疑问，请确认
'无条件共享，作为政务信息公开。': '无条件开放',
'作为行政依据、业务参考': '无条件开放',
'眉山市市级示范社名录': '无条件开放',  # 有疑问，请确认
'政务信息主动公开流程图': '无条件开放',  # 有疑问，请确认
'个体工商户变更登记流程图': '无条件开放',  # 有疑问，请确认
'个体工商户开业登记流程图': '无条件开放',  # 有疑问，请确认
'个体工商户注销登记流程图': '无条件开放',  # 有疑问，请确认
'名称争议事项流程图': '无条件开放',  # 有疑问，请确认
'有业务需求的进行共享': '有条件开放',
'信访条例': '无条件开放',
'预算法': '无条件开放',
'无条件贡献': '无条件开放',
'作为行政依据，工作参考。': '无条件开放',
'龙马镇商贸行业燃气安全隐患管理台账': '无条件开放',  # 有疑问，请确认
'依条件申请': '有条件开放',
'重点服务业企业培育情况表': '无条件开放',  # 有疑问，请确认
'需向部门申请': '有条件开放',
'政务信息资源': '无条件开放',
'区科协科普e展点位': '无条件开放',  # 有疑问，请确认
'彭山区农技协开展培训情况': '无条件开放',  # 有疑问，请确认
'需要可自行下载': '无条件开放',
'开发共享下载': '无条件开放',
'开发下载': '无条件开放',
'开放下载': '无条件开放',
'可作为工作参考': '无条件开放',
'需求部门申请共享': '有条件开放',
'无条件分析': '无条件开放',
'廉政法规知识测试资料': '无条件开放',
'丹棱县村级小微权力清单': '无条件开放',  # 有疑问，请确认
'丹棱县纪委年度部门预算': '无条件开放',  # 有疑问，请确认
'群众信访举报途径电子表格数据': '无条件开放',  # 有疑问，请确认
'用于档案数据校验': '无条件开放',
'地方政府专项债券分区域限额': '无条件开放',  # 有疑问，请确认
'用于档案查阅利用': '无条件开放',
'申请同意后开放共享': '有条件开放',
'眉山市公共资源交易领域信息主动公开目录': '无条件开放',  # 有疑问，请确认
'需要可以出函申请': '有条件开放',
'共享平台': '无条件开放',
'需个人或单位出具查档介绍信或查档函': '有条件开放',
'获奖名单': '无条件开放',  # 有疑问，请确认
'招生依据': '无条件开放',  # 有疑问，请确认
'用于数据校核': '无条件开放',
'非盈利性使用': '无条件开放',
'无条件开发1': '无条件开放',  # 有疑问，请确认
'查询人基本信息，身份证信申请查询': '有条件开放',
'待县人大批准2020年部门决算后开放': '有条件开放',
'县人大批准2020年部门决算后开放': '有条件开放',
'无条件共享开放': '无条件开放',
'眉山司法局香港、澳门永久性居民中的中国居民申请在眉从事律师职业核准办事指南': '无条件开放',  # 有疑问，请确认
'中华人民共和国公证法': '无条件开放',
'眉山市司法局兼职律师执业许可办事指南': '无条件开放',  # 有疑问，请确认
'眉山市司法局律师事务所设立办事指南': '无条件开放',  # 有疑问，请确认
'眉山市司法局关于设立律师事务所（分所）的指南': '无条件开放',  # 有疑问，请确认
'眉山市司法局法律援助律师执业许可办事指南': '无条件开放',  # 有疑问，请确认
'眉山市司法局台湾居民申请在川从事律师职业许可办事指南': '无条件开放',  # 有疑问，请确认
'眉山市司法局专职律师执业许可办事指南': '无条件开放',  # 有疑问，请确认
'眉山公证机构及公证员基本信息': '无条件开放',  # 有疑问，请确认
'共享类文件': '无条件开放',
'农村居民可支配收入与支出': '无条件开放',  # 有疑问，请确认
'2020年丹棱县本级一般公共预算支出预算表': '无条件开放',  # 有疑问，请确认
'2020年丹棱县政府性基金收入预算表': '无条件开放',  # 有疑问，请确认
'填写申请后开放': '有条件开放',
'有条件共享（申请）': '有条件开放',
'登录公开': '无条件开放',
'按相关规定审核开放共享': '有条件开放',
'按需审批使用': '有条件开放',
'按需审核使用': '有条件开放',
'用于数据校验': '无条件开放',
'垃圾分类等场景': '无条件开放',  # 有疑问，请确认
'按相关政策要求，当前数据允许开放': '无条件开放',
'招聘会信息，无条件开放': '无条件开放',
'无条件开': '无条件开放',
'受限开放': '有条件开放',
'用于数据查看与应用': '无条件开放',
'因城市管理、综合执法需要可使用本数据': '无条件开放',
'按需开放': '有条件开放',
'开放网站': '无条件开放',
'申请共享': '有条件开放',
'开放网站申请': '有条件开放',
'根据接口规范': '无条件开放',
'1级\t不脱敏': '无条件开放',  # 有疑问，请确认
'依据申请': '有条件开放', '需要审批': '有条件开放',
"('无条件开放',)": '无条件开放', "用户无需登录即可下载数据": '无条件开放',
"('有条件开放',)": '有条件开放', '用于工作参考': '有条件开放',
"用户须用“浙里办”账号登录后实名下载数据": '无条件开放', "无": '无条件开放', '无条件共享工作参考': '无条件开放', '暂无': '无条件开放', '作为工作参考': '无条件开放'
}

df_copy['open_conditions'] = df_copy['open_conditions'].apply(map_open_conditions)
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - open_conditions 列已映射完成")
print("以下是无法映射的open_conditions:", open_condition_undefined)

df_copy['is_api'] = df_copy['is_api'].apply(convert_to_bool)
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - is_api 列已转换为布尔值")

df_copy['subject'] = df_copy['subject'].fillna('无分类主题')
df_copy['subject'] = df_copy['subject'].str.replace(" ", "")
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 'subject' 列空值填充完成")

df_copy['source_department'] = df_copy['source_department'].fillna('无来源部门')
# 替换掉“【】”以及里面的内容
df_copy['source_department'] = df_copy['source_department'].str.replace(r'【.*?】', '', regex=True)
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} -'source_department' 列空值填充完成")

df_copy['title'] = df_copy['title'].fillna('无标题')
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 'title' 列空值填充完成")

# 添加省份编号列
df_copy['城市编号'] = df_copy['location'].map(city_dict)


# 将chosen_time列转换为日期时间格式
df_copy['chosen_time'] = pd.to_datetime(df_copy['chosen_time'], errors='coerce')
# 分组并找到每个分组内chosen_time最早的值
df_copy['数据初次发布时间'] = df_copy.groupby('location')['chosen_time'].transform('min').dt.strftime('%Y-%m-%d')

2025-03-06 16:13:20 - 数据容量转换完成
2025-03-06 16:13:42 - 定时更新率计算完成
以下是未知的更新周期： {'无条件共享': 29, '每年1': 1, '1': 93, '数据来源周期不确定': 1, '已归档，不再更新': 4, '根据实际情况更新': 45, '根据统计结果更新': 1, '根据项目建设情况更新': 1, '根据办理情况更新': 2, '根据处罚情况更新': 1, '有新执业人员时更新': 1, '根据实际变化情况和上级要求进行更新': 1, '市级司法行政机关负责初审，省 级司法行政机关最终审核': 1, '评选表彰先进集体需市委市政府统筹批准后方可实施': 1, '按照机构改革后公布': 1, '非实时更新事项': 1, '公证机构执业信息每年更新一次': 1, '规范性文件出台需履行相关程序': 1, '省司法厅统一组织培训考试': 1, '按单位规定更新': 10, '不定更新': 2, '按单位要求更新': 6, '依据工作开展情况更新': 1, '根据项目申请情况更新': 1, '根据规划项目的进展情况': 1, '较长时间评一次': 2, '较长时间调整一次': 1, '按照数据产生频率及时更新相关数据': 17, '根据业务实际更新': 1, '根据业务量产生数据': 1, '根据通报随时更新': 1, '根据工作情况调整': 1, '每年举办一次': 2, '视业务办理频率而定': 1, '每年按照省厅安排进行评选，2019年暂停，后续情况视省住建厅安排。': 1, '数据在省平台产生': 6, '权力移交市行政审批服务局，本单位仅有移交前的历史数据。': 3, '按照本市计划而定': 1, '按照省计划开展': 1, '2020年暂停，后续情况视省住建厅安排。': 1, '数据在国家平台产生': 1, '按照单位自定义更新': 1, '不定时收集信息': 7, '业务量较少': 4, '数据不定时': 1, '社保转移从转出到办结有时限': 1, '需要企业提交招聘信息': 1, '计算机考试每年一次': 1, '德州市主要河流水情历史信息不定期统计': 1, '德州市供水量非实时统计': 1, '取用水户测点信息没有变化不需要实时更新': 1, '用水户不变无法实时更新': 1,

In [34]:
# 打印不在映射中的开放条件及其对应的标题和位置
def print_unmapped_open_conditions(df):
    """
    打印所有不在open_conditions_mapping中的开放条件及其对应的标题和位置
    
    参数:
    df (DataFrame): 包含open_conditions, title, location列的数据框
    """
    # 确保必要的列存在
    required_cols = ['open_conditions', 'title', 'location']
    if not all(col in df.columns for col in required_cols):
        print("数据框缺少必要的列：open_conditions, title, location")
        return
    
    # 创建一个新的DataFrame，只包含不在映射中的记录
    unmapped_conditions = []
    
    for _, row in df.iterrows():
        condition = row['open_conditions']
        # 跳过空值
        if pd.isnull(condition) or condition == '' or condition == 'nan':
            continue
            
        # 如果是字符串，去除中文标点
        if isinstance(condition, str):
            condition = remove_chinese_punctuation(condition)
        
        # 检查是否在映射中
        if condition not in open_conditions_mapping:
            unmapped_conditions.append({
                'location': row['location'],
                'title': row['title'],
                'open_conditions': condition
            })
    
    # 创建DataFrame并显示
    if unmapped_conditions:
        unmapped_df = pd.DataFrame(unmapped_conditions)
        print(f"找到 {len(unmapped_df)} 条不在映射中的开放条件记录")
        
        # 按开放条件分组并计数
        condition_counts = unmapped_df['open_conditions'].value_counts()
        print("\n开放条件出现频率:")
        print(condition_counts.head(20))  # 显示前20个最常见的未映射条件
        
        # 显示详细记录
        print("\n详细记录示例(前20条):")
        pd.set_option('display.max_colwidth', 100)  # 设置列宽以便更好地显示
        print(unmapped_df.head(20))
        
        # 保存到Excel文件
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f'../../output/unmapped_open_conditions_{timestamp}.xlsx'
        unmapped_df.to_excel(output_file, index=False)
        print(f"\n完整结果已保存至: {output_file}")
    else:
        print("未找到不在映射中的开放条件")

# 调用函数
print_unmapped_open_conditions(df_copy)

找到 405 条不在映射中的开放条件记录

开放条件出现频率:
None                                               116
提供查询服务                                              34
涉及个人信息商业秘密保密商务信息的公共数据指向的特定自然人法人或者非法人组织依法授权同意开放的     30
涉及个人信息的公共数据经匿名化处理的                                  16
每年                                                  14
涉及商业秘密保密商务信息的公共数据经脱敏脱密处理的                           11
用于工作参考/用于数据校验等                                      10
申请人说明数据应用场景用途等数源部门根据申请理由进行审核通过后方可使用                 10
涉密数据已脱敏脱密                                            9
每月                                                   7
提供查询的函                                               7
仅用于工作参考数据校核                                          5
开放网站,无                                               5
不敏感                                                  5
网站开放                                                 4
仅内部使用                                                3
工作需要                                                 3
根据应用场景受限开放                       

In [29]:
def safe_sum(x):
    """
    计算非空值的总和
    :param x: 包含数值的列表或Series
    :return: 非空值的总和，如果所有值都为空，则返回None
    """
    valid_values = [volume for volume in x if pd.notna(volume)]
    return sum(valid_values) if valid_values else None

def safe_mean(x):
    """
    计算非空值的平均值
    :param x: 包含数值的列表或Series
    :return: 非空值的平均值，如果所有值都为空，则返回None
    """
    valid_values = [value for value in x if pd.notna(value)]
    return sum(valid_values) / len(valid_values) if valid_values else None

def safe_condition_mean(condition):
    """
    计算满足条件的非空值的比例
    :param condition: 条件值
    :return: 一个函数，该函数计算满足条件的非空值的比例，如果所有值都为空，则返回None
    """
    def inner(x):
        valid_values = [value for value in x if pd.notna(value)]
        count_condition = sum(1 for value in valid_values if value == condition)
        return count_condition / len(valid_values) if valid_values else None
    return inner

def proportion_greater_than_zero(series):
    if series.isna().all():
        return None
    
    return (series > 0).mean()


In [30]:
# 确保数据帧按照分组键排序
df_result = df_copy.sort_values(by=['location', 'source_department', 'subject', 'year'])

# 设置分组键为索引
df_result = df_result.set_index(['location', 'source_department', 'subject', 'year'])

# 分组统计各项数据
grouped = df_result.groupby(level=[0, 1, 2, 3])

In [31]:
summary = grouped.agg({
    'title': 'count',  # 文本数
    'data_volume': safe_sum,  # 总数据量
    '定时更新率': proportion_greater_than_zero,  # 大于0的值的占比
    'is_update': safe_mean,  # 更新文本率
    'open_conditions': [
        safe_condition_mean('无条件开放'),  # 无条件开放文本率
        safe_condition_mean('有条件开放')  # 有条件开放文本率
    ],
    'is_api': safe_mean,  # 有接口文本率
    'access_count': safe_sum,  # 总访问次数
    'download_count': safe_sum,  # 总下载次数
    '城市编号': 'first',  # 省份编号
    '数据初次发布时间': 'first',  # 数据初次发布时间
    'province': 'first',  # 省份名称
}).reset_index()

# 重命名统计列
summary.columns = ['地区', '来源部门', '主题', '年份', '文本数', '总数据量', '定时更新率', '更新文本率', '无条件开放文本率', '有条件开放文本率', '有接口文本率', '总访问次数', '总下载次数', '城市编号', '该地区数据初次发布时间', '省份']

# 重新排列列顺序
new_column_order = ['省份', '地区', '城市编号', '该地区数据初次发布时间', '来源部门', '主题', '年份', '文本数', '总数据量', '定时更新率', '更新文本率', '无条件开放文本率', '有条件开放文本率', '有接口文本率', '总访问次数', '总下载次数']
summary = summary.reindex(columns=new_column_order)

# 保存统计信息到Excel文件
summary.to_excel(f'{output_path}/市级统计信息汇总_{time.strftime("%Y%m%d_%H%M%S")}.xlsx', index=False)

print("统计信息已保存至:", f'{output_path}/市级统计信息汇总_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')


C:\Users\dass1\AppData\Local\Temp\ipykernel_18996\4228361939.py:16: PerformanceWarning: indexing past lexsort depth may impact performance.
  }).reset_index()
C:\Users\dass1\AppData\Local\Temp\ipykernel_18996\4228361939.py:16: PerformanceWarning: indexing past lexsort depth may impact performance.
  }).reset_index()
C:\Users\dass1\AppData\Local\Temp\ipykernel_18996\4228361939.py:16: PerformanceWarning: indexing past lexsort depth may impact performance.
  }).reset_index()


统计信息已保存至: ../../docs/市级统计信息汇总_20250306_162321.xlsx


In [33]:
# df_copy.to_csv('./test.csv')